In [1]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('./SD-Datasets')

Mounted at ./SD-Datasets


In [3]:
train_dir = './SD-Datasets/MyDrive/SD-Datasets/Skin Disease_dataset/TRAIN' #ini gdrive pribadi resti
test_dir = './SD-Datasets/MyDrive/SD-Datasets/Skin Disease_dataset/TEST'

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 800 images belonging to 10 classes.
Found 200 images belonging to 10 classes.


In [5]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

58889256/58889256 [==============================] - 0s 0us/step


In [6]:
for layer in base_model.layers[-4:]:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history_finetune = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/10
25/25 [==============================] - 867s 34s/step - loss: 2.4281 - accuracy: 0.1037 - val_loss: 2.2827 - val_accuracy: 0.1302
Epoch 2/10
25/25 [==============================] - 839s 34s/step - loss: 2.3600 - accuracy: 0.1338 - val_loss: 2.2353 - val_accuracy: 0.2031
Epoch 3/10
25/25 [==============================] - 852s 34s/step - loss: 2.2649 - accuracy: 0.1637 - val_loss: 2.1838 - val_accuracy: 0.3073
Epoch 4/10
25/25 [==============================] - 827s 33s/step - loss: 2.1631 - accuracy: 0.1838 - val_loss: 2.1633 - val_accuracy: 0.3021
Epoch 5/10
25/25 [==============================] - 832s 33s/step - loss: 2.0771 - accuracy: 0.2475 - val_loss: 2.0943 - val_accuracy: 0.3073
Epoch 6/10
25/25 [==============================] - 835s 34s/step - loss: 1.9963 - accuracy: 0.2575 - val_loss: 2.0712 - val_accuracy: 0.3385
Epoch 7/10
25/25 [==============================] - 834s 34s/step - loss: 1.8812 - accuracy: 0.2887 - val_loss: 1.9428 - val_accuracy: 0.3281
Epoch 